TODO: waarschijnlijk verwijderen
TODO: na kijken of echt alles om is gezet in andere files voordat het werwijderd word.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from collections import defaultdict

In [ ]:
chosen_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Czechia', 'Estonia', 'Finland', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Korea', 'Latvia', 'Luxembourg', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Türkiye']

life_expectancy_data = pd.read_csv('../data/life_expectancy.csv')
life_vars = life_expectancy_data["Variable"].unique().tolist()
pharma_sales_data = pd.read_csv('../data/pharma_sales_ppp.csv')
drug_vars = pharma_sales_data["Variable"].unique().tolist()

In [ ]:
def merge_and_plot(filtered_data_x, filtered_data_y, label_x, label_y):
    merged_data = pd.merge(filtered_data_x, filtered_data_y, on="Country", how="inner")
    selected_data = merged_data[["Country", "Value_x", "Value_y"]]
    selected_data.columns = ["Country", label_x, label_y]

    # .reshape(X.shape[0])
    x_data = np.array(selected_data[label_x])
    y_data = np.array(selected_data[label_y])
    print(np.log(x_data), np.log(y_data))
    # plt.scatter(np.log(np.array(selected_data[label_x])/(1 - np.array(selected_data[label_x]))), np.log(np.array(selected_data[label_y])/(1 - np.array(selected_data[label_y]))))
    # plt.xlabel(label_x)
    # plt.ylabel(label_y)
    # plt.show()

def p_value_correlation(filtered_data_x, filtered_data_y, label_x, label_y, threshold):

    merged_data = pd.merge(filtered_data_x, filtered_data_y, on="Country", how="inner")
    selected_data = merged_data[["Country", "Value_x", "Value_y"]]
    selected_data.columns = ["Country", label_x, label_y]
    corr, p_value = spearmanr(selected_data[label_x], selected_data[label_y])

    if corr >= threshold or corr <= -threshold:

        # p value calculated in the spearman correlation is the correlation value
        # it is first transformed to a Z-score which normalizes it, so the data can
        # be seen as a normal distribution. The p-value is calculated with the cdf
        # and does a two tailed test. The test is done with a t-distribution
        print("X:", label_x, "Y:", label_y)
        print("correlation between X and Y: ", corr)
        print("p value for correlation: ", p_value)

        if p_value <= 0.05:
            print("HYPOTHESIS REJECTED")
        else:
            print("HYPOTHESIS ACCEPTED")

def filter_data(data, year, variable, measure=None):
    if measure:
        return data[
            (data["Year"] == year) &
            (data["Country"].isin(chosen_countries)) &
            (data["Variable"] == variable) &
            (data["Measure"] == measure)
        ]
    else:
        return data[
            (data["Year"] == year) &
            (data["Country"].isin(chosen_countries)) &
            (data["Variable"] == variable)
        ]


In [ ]:
for drug in drug_vars:

    filtered_pharma_sales = filter_data(pharma_sales_data, 2014, drug)
    for life in life_vars:
        filtered_life_expectancy = filter_data(life_expectancy_data, 2014, life, "Years")

        merge_and_plot(filtered_pharma_sales, filtered_life_expectancy, str(drug), str(life))
        p_value_correlation(filtered_pharma_sales, filtered_life_expectancy, str(drug), str(life), 0.4)


In [ ]:


for drug1 in drug_vars:

    filtered_pharma_sales1 = filter_data(pharma_sales_data, 2014, drug1)

    for drug2 in drug_vars:
        filtered_pharma_sales2 = filter_data(pharma_sales_data, 2014, drug2)
        if drug1 != drug2:

            # merge_and_plot(filtered_pharma_sales1, filtered_pharma_sales2, str(drug1), str(drug2))
            p_value_correlation(filtered_pharma_sales1, filtered_pharma_sales2, str(drug1), str(drug2), 0.4)

In [ ]:
alcohol_consumption_data = pd.read_csv('../data/alcohol_consump.csv')
alcohol_measures = alcohol_consumption_data["Measure"].unique().tolist()

for alcohol in alcohol_measures:
    filtered_alcohol_consumption = filter_data(alcohol_consumption_data, 2014, "Alcohol consumption", alcohol)

    for life in life_vars:
        filtered_life_expectancy = filter_data(life_expectancy_data, 2014, life, "Years")

        # merge_and_plot(filtered_alcohol_consumption, filtered_life_expectancy, str(alcohol), str(life))
        p_value_correlation(filtered_alcohol_consumption, filtered_life_expectancy, str(alcohol), str(life), 0.4)

In [ ]:
# food data werkt nog niet

food_data = pd.read_csv('../data/food.csv')
food_measures = food_data["Measure"].unique().tolist()
food_variables = food_data["Variable"].unique().tolist()

for food in food_variables:
    filtered_food_data = filter_data(food_data, 2014, food)

    for life in life_vars:
        filtered_life_expectancy = filter_data(life_expectancy_data, 2014, life, "Years")


        merge_and_plot(filtered_food_data, filtered_life_expectancy, str(food), str(life))
        p_value_correlation(filtered_food_data, filtered_life_expectancy, str(food), str(life), 0.4)

Testing for heteroscedasticity with the multiple regression for the genders apart. In the plots a coneshape can be seen denk ik.

In [ ]:
from sklearn.linear_model import LinearRegression

variables = ["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)", "N-Nervous system", "N06A-Antidepressants"]
genders = ["Females at age 40", "Males at age 40"]


filtered_pharma_sales = pharma_sales_data[
    (pharma_sales_data["Year"] == 2014) &
    (pharma_sales_data["Country"].isin(chosen_countries)) &
    (pharma_sales_data["Variable"].isin(variables))
]

df = defaultdict(lambda: [])
for drug in filtered_pharma_sales[["Variable", "Value"]].iterrows():
    df[drug[1]["Variable"]].append(drug[1]["Value"])

df = pd.DataFrame(df)

X = df[["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)", "N-Nervous system", "N06A-Antidepressants"]]

for gender in genders:
    filtered_life_expectancy = filter_data(life_expectancy_data, 2014, gender, "Years")

    y = filtered_life_expectancy["Value"]
    model = LinearRegression()
    model.fit(X, y)
    intercept = model.intercept_
    coeff = model.coef_
    print("the intercept: ", intercept, "the coefficients: ", coeff)
    y_predicted = model.predict(X)
    residuals = y - y_predicted

    print("sum of the residuals: ", np.sum(residuals))
    plt.xlabel("de 'echte' waarde van leeftijd " + gender)
    plt.scatter(y, residuals)
    plt.ylabel("de residuals voor die 'echte' waarde")
    plt.show()


Checking for heteroskedacity and doing regression on one of the variables and visualizing the data.

In [ ]:
def remove_outliers_and_refit(df, target, feature, num_std_devs=4):
    X = df[[feature]].reset_index(drop=True)
    y = target.reset_index(drop=True)
    outliers_removed = True

    while outliers_removed:
        # Fit the model
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        residuals = y - y_pred

        # Calculate standard deviation of residuals
        std_dev = np.std(residuals)

        # Identify non-outliers
        non_outlier_mask = np.abs(residuals) <= num_std_devs * std_dev
        prev_len = len(X)
        X = X[non_outlier_mask.values]
        y = y[non_outlier_mask]

        # Check if any outliers were removed in this iteration
        outliers_removed = len(X) < prev_len

    # Refit the model with the final cleaned data
    model.fit(X, y)

    return model, X, y

In [ ]:
for gender in genders:
    for drug in variables:

        X = df[[drug]]
        X_array = np.array(X).reshape(X.shape[0])

        target = np.exp(filter_data(life_expectancy_data, 2014, gender, "Years")["Value"])

        plt.scatter(X_array, np.array(target))
        plt.title(gender + drug + "exp plot")
        plt.xlabel(drug)
        plt.ylabel(gender)
        plt.show()

        model, X_cleaned, y_cleaned = remove_outliers_and_refit(df, target, drug, 3)

        plt.scatter(X_cleaned, y_cleaned)
        y_pred_cleaned = model.predict(X_cleaned)
        plt.plot(X_cleaned, y_pred_cleaned, color="orange", label='Final Fitted Line')
        plt.title(f"{gender} - {drug} (Final Cleaned Data)")
        plt.xlabel(drug)
        plt.ylabel(f"{gender} Life Expectancy")
        plt.legend()
        plt.show()

        print(f"Final Intercept: {model.intercept_}, Final Coefficient: {model.coef_[0]}")

        residuals = y_cleaned - y_pred_cleaned
        # print(residuals)
        plt.scatter(y_cleaned, y_pred_cleaned)
        plt.ylim(plt.xlim())
        plt.xlabel("cleaned true life expectancy")
        plt.ylabel("cleaned predicted life expectancy")
        plt.show()

        plt.scatter(y_cleaned, residuals)
        plt.xlabel("true life expectancy")
        plt.ylabel("residuals y_cleaned - y_pred_cleaned")
        plt.show()

        # x_pharmasales = np.linspace(min(X[drug]), max(X[drug]), 26)

        # plt.scatter(X, y)
        # plt.xlabel('ppp van' + drug)
        # plt.ylabel(gender)
        # plt.plot(x_pharmasales, intercept + x_pharmasales * coeff, color="orange", label='fitted line')
        # plt.legend()
        # plt.show()


In [ ]:
filtered_pharma_sales = pharma_sales_data[
    (pharma_sales_data["Year"] == 2014) &
    (pharma_sales_data["Country"].isin(chosen_countries)) &
    (pharma_sales_data["Variable"].isin(variables))
]

df = defaultdict(lambda: [])
for drug in filtered_pharma_sales[["Variable", "Value"]].iterrows():
    df[drug[1]["Variable"]].append(drug[1]["Value"])

df = pd.DataFrame(df)


df["interaction_1"] = df["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)"] * df["N-Nervous system"]
df["interaction_2"] = df["N06A-Antidepressants"] * df["N-Nervous system"]
df["interaction_3"] = df["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)"] * df["N06A-Antidepressants"]
X = df[["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)", "N-Nervous system", "N06A-Antidepressants", "interaction_1", "interaction_2", "interaction_3"]]



for gender in genders:

    y = np.log(filter_data(life_expectancy_data, 2014, gender, "Years")["Value"])
    X = np.log(X)

    model = LinearRegression()
    model.fit(X, y)
    intercept = model.intercept_
    coeff = model.coef_

    print("the intercept: ", intercept, "the coefficient: ", coeff)
    y_pred = model.predict(X)

    residuals = y - y_pred


    y = np.exp(y)
    y_pred = np.exp(y_pred)
    plt.scatter(y, y_pred)
    plt.ylim(plt.xlim())
    plt.xlabel("true life expectancy")
    plt.ylabel("predicted life expectancy")
    plt.title(gender)
    plt.show()

    plt.scatter(y, residuals)
    plt.xlabel("true life expectancy")
    plt.ylabel("residuals true y - y predicted")
    plt.title(gender)
    plt.show()